In [1]:
import logging

# 로깅 설정 - pdfminer 경고 메시지 억제
logging.basicConfig(level=logging.ERROR)
logging.getLogger('pdfminer').setLevel(logging.ERROR)

In [2]:
import os
import logging

# 로깅 설정 - pdfminer 경고 메시지 억제
logging.basicConfig(level=logging.ERROR)
logging.getLogger('pdfminer').setLevel(logging.ERROR)

from langchain_community.document_loaders import PDFMinerLoader, PyMuPDFLoader

# PDFMinerLoader와 PyMuPDFLoader 둘다 시도
def PDF_Loader(pdf_file):
    loaders = [
        ("PDFMinerLoader", lambda: PDFMinerLoader(pdf_file, mode="page")),
        ("PyMuPDFLoader", lambda: PyMuPDFLoader(pdf_file))
    ]
    
    for loader_name, loader_func in loaders:
        try:
            loader = loader_func()
            docs = loader.load()
            print(f"성공: {os.path.basename(pdf_file)} - {loader_name} 사용")
            return docs
        except Exception as e:
            error_msg = str(e).lower()
            # 색상 관련 오류인지 확인
            if any(keyword in error_msg for keyword in ['color', 'stroke', 'gray', 'rgb', 'cmyk', 'float']):
                print(f"색상 오류 발생: {os.path.basename(pdf_file)} - {loader_name}에서 {e}")
                continue
            else:
                print(f"기타 오류 발생: {os.path.basename(pdf_file)} - {loader_name}에서 {e}")
                continue

    print(f"모든 로더 실패: {os.path.basename(pdf_file)}")
    return []

In [3]:
pdf_dir = "./Data"
pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.lower().endswith('.pdf')]
print(f"Total PDF files: {len(pdf_files)}")

# 문서 로드
docs = []
success_count = 0
error_count = 0

for i,pdf_file in enumerate(pdf_files):
    if i % 100 == 0:
        print(f"Processing file {i+1} of {len(pdf_files)}: {pdf_file}")
    file_docs = PDF_Loader(pdf_file)
    if file_docs:
        docs.extend(file_docs)
        success_count += 1
    else:
        error_count += 1

print(f"\n=== 처리 결과 ===")
print(f"성공적으로 로드된 파일 수: {success_count}")
print(f"로드 실패한 파일 수: {error_count}")
print(f"총 로드된 문서 수: {len(docs)}")

Total PDF files: 5
Processing file 1 of 5: ./Data/465426.pdf
성공: 465426.pdf - PDFMinerLoader 사용
성공: 306694.pdf - PDFMinerLoader 사용
성공: 613523.pdf - PDFMinerLoader 사용
성공: 2015_PROF_311_Macular-hole-surgery-dataset(황반구멍).pdf - PDFMinerLoader 사용
성공: 218608.pdf - PDFMinerLoader 사용

=== 처리 결과 ===
성공적으로 로드된 파일 수: 5
로드 실패한 파일 수: 0
총 로드된 문서 수: 116


In [4]:
from utils.docs_storage_utils import save_docs_to_local, load_docs_from_local

save_docs_to_local(docs, save_dir="./Data/saved_docs", filename="filename.pkl")
docs = load_docs_from_local("./Data/saved_docs/filename.pkl")

문서가 성공적으로 저장되었습니다: ./Data/saved_docs/filename.pkl
저장된 문서 수: 116
문서가 성공적으로 로드되었습니다: ./Data/saved_docs/filename.pkl
로드된 문서 수: 116


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False
)
docs = text_splitter.split_documents(docs)

print(f"분할 후 문서 수: {len(docs)}")

분할 후 문서 수: 552


In [9]:
import pickle
from langchain.retrievers import BM25Retriever

# BM25Retriever 생성
bm25_retriever = BM25Retriever.from_documents(
    docs,
)
bm25_retriever.k = 2 

# 검색 실행
# results = bm25_retriever.get_relevant_documents("what is epiretinal membrane?")
# for result in results:
#     print(f"문서: {result.page_content}")
#     print(f"소스: {result.metadata['source']}")
#     print("---")

# BM25Retriever를 로컬에 저장
with open("./Data/bm25/bm25_retriever.pkl", "wb") as f:
    pickle.dump(bm25_retriever, f)

with open("./Data/bm25/bm25_retriever.pkl", "rb") as f:
    loaded_bm25_retriever = pickle.load(f)

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embedding_model_name = "ibm-granite/granite-embedding-125m-english"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"device": "mps"},
    encode_kwargs={"normalize_embeddings": True}
)

# print(hf_embeddings.embed_documents("hi"))
print(len(hf_embeddings.embed_query("hi"))) # 768

In [ ]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents=docs, embedding=hf_embeddings)
db.save_local(folder_path="./Data/faiss_db", index_name="Ophthalmology_FAISS_Index")


In [ ]:
loaded_db = FAISS.load_local(
    folder_path="./Data/faiss_db", 
    index_name="Ophthalmology_FAISS_Index",
    embeddings=hf_embeddings,
    allow_dangerous_deserialization=True,
    )

retriever = loaded_db.as_retriever(
    search_type="mmr", 
    search_kwargs={"k": 4, "lambda_mult": 0.25, "fetch_k": 10},
    )

retrieved_docs = retriever.invoke("What is Epiretinal Membrane?")

for i in range(len(retrieved_docs)):
    print(f"Document {i+1}:")
    print(retrieved_docs[i].page_content)
    print("-"*100)